# Jリーグ観客動員数予測コンテスト

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
import plotly.plotly as py
import plotly.graph_objs as go
import plotly
plotly.offline.init_notebook_mode(connected=True)
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from pandas.tseries.offsets import *
%matplotlib inline

### データ読込

In [2]:
train = pd.read_csv("train.csv" )
test = pd.read_csv("test.csv") 
holidays_in_japan = pd.read_csv("holidays_in_japan.csv" )
match_reports= pd.read_csv("match_reports.csv" )
stadium_capacity_mapping= pd.read_csv("stadium_capacity_mapping.csv" )

In [3]:
print(train.head())
print(train.shape)

     id  match_date kick_off_time section round home_team away_team  \
0  9190  2006-03-04         16:04     第1節   第1日       G大阪        浦和   
1  9191  2006-03-05         13:00     第1節   第2日        甲府        清水   
2  9192  2006-03-05         13:35     第1節   第2日      FC東京        大分   
3  9193  2006-03-05         14:04     第1節   第2日        磐田        福岡   
4  9194  2006-03-05         14:04     第1節   第2日       名古屋       C大阪   

              venue weather  temperature  humidity  \
0           万博記念競技場       晴          8.3        40   
1  山梨県小瀬スポーツ公園陸上競技場       晴         12.9        28   
2          味の素スタジアム       晴         12.1        35   
3        静岡スタジアムエコパ       晴         11.6        42   
4       名古屋市瑞穂陸上競技場       晴         13.1        32   

                                 broadcasters  attendance  
0                           NHK総合/J SPORTS(録)       20916  
1                   山梨放送/テレビ静岡(録)/J SPORTS(録)       14277  
2                   BS-i/MXテレビ(録)/J SPORTS(録)       22531  
3       

In [4]:
print(test.head())
print(test.shape)

      id  match_date kick_off_time section round home_team away_team  \
0  19075  2017-02-25         12:35     第1節   第1日      横浜FM        浦和   
1  19076  2017-02-25         14:04     第1節   第1日        仙台        札幌   
2  19077  2017-02-25         14:04     第1節   第1日        鹿島      FC東京   
3  19078  2017-02-25         14:04     第1節   第1日        清水        神戸   
4  19079  2017-02-25         14:04     第1節   第1日        広島        新潟   

            venue weather  temperature  humidity        broadcasters  
0         日産スタジアム       晴         10.2      45.0     DAZN/テレビ神奈川/テレ玉  
1    ユアテックスタジアム仙台       曇         10.6      36.0    DAZN/NHK仙台/NHK札幌  
2  県立カシマサッカースタジアム       晴         10.1      63.0          DAZN/NHK総合  
3     IAIスタジアム日本平       晴         15.0      40.0  DAZN/NHK BS1/テレビ静岡  
4    エディオンスタジアム広島       晴         11.2      45.0    DAZN/NHK広島/NHK新潟  
(477, 12)


In [5]:
#train,testを統合
compose = pd.concat((train,test),axis=0)
compose = compose.reset_index().copy()
print(train.shape)
print(test.shape)

(3366, 13)
(477, 12)


In [6]:
print(compose.head())
print(compose.shape)

   index  attendance away_team                                broadcasters  \
0      0     20916.0        浦和                           NHK総合/J SPORTS(録)   
1      1     14277.0        清水                   山梨放送/テレビ静岡(録)/J SPORTS(録)   
2      2     22531.0        大分                   BS-i/MXテレビ(録)/J SPORTS(録)   
3      3     28564.0        福岡                                    J SPORTS   
4      4     17199.0       C大阪  スカイパーフェクTV!/NHK名古屋(録)/NHK大阪(録)/J SPORTS(録)   

  home_team  humidity    id kick_off_time  match_date round section  \
0       G大阪      40.0  9190         16:04  2006-03-04   第1日     第1節   
1        甲府      28.0  9191         13:00  2006-03-05   第2日     第1節   
2      FC東京      35.0  9192         13:35  2006-03-05   第2日     第1節   
3        磐田      42.0  9193         14:04  2006-03-05   第2日     第1節   
4       名古屋      32.0  9194         14:04  2006-03-05   第2日     第1節   

   temperature             venue weather  
0          8.3           万博記念競技場       晴  
1         12.9  山梨

欠損データの確認

In [7]:
# 欠損データの確認 missing_values_tableを作る
def missing_values_table(df):
    mis_val = df.isnull().sum()
    mis_val_percent = 100 * df.isnull().sum()/len(df)
    mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
    mis_val_table_ren_columns = mis_val_table.rename(
    columns= {0 : 'Missing Values', 1: '% of total values'})
    return mis_val_table_ren_columns

In [8]:
missing_values_table(compose)

,Missing Values,% of total values
index,0,0.000000
attendance,477,12.412178
away_team,0,0.000000
broadcasters,0,0.000000
home_team,0,0.000000
humidity,18,0.468384
id,0,0.000000
kick_off_time,0,0.000000
match_date,0,0.000000
round,0,0.000000


### attendanceは今回予測するデータ 他に気温・湿度・天候にわずかながらNanデータが存在する
なので,欠損データは今回平均で置き換えることにした　https://newtechnologylifestyle.net/pandasnan/
精査必要あり（10/21）

In [9]:
#欠損値を平均で置き換える
compose["humidity"]    = compose["humidity"].fillna(compose["humidity"].mean())
compose["temperature"] = compose["temperature"].fillna(compose["temperature"].mean())
#compose_processing["weather"]   = compose_processing["weather"].fillna(compose_processing["weather"].mean())

In [10]:
holidays_in_japan 
#日本の祝日　祝日は来場者が多いと予想される。　フラグをたてる事を考える

,holiday_date,description
0,2006-01-01,New Years Day
1,2006-01-02,New Years Holiday
2,2006-01-03,New Years Holiday
3,2006-01-09,Coming-of-age Day
4,2006-02-11,National Foundation Day
5,2006-03-21,Vernal Equinox Day
6,2006-04-29,Greenery Day
7,2006-05-03,Constitution Memorial Day
8,2006-05-04,National Holiday
9,2006-05-05,Childrens Day


In [11]:
match_reports  
#人気選手がいるかもしれない...
#データ数が多くなるのでとりあえず保留

,id,home_team_player11,home_team_player10,home_team_player9,home_team_player8,home_team_player7,home_team_player6,home_team_player5,home_team_player4,home_team_player3,...,away_team_player2,away_team_player3,away_team_player4,away_team_player5,away_team_player6,away_team_player7,away_team_player8,away_team_player9,away_team_player10,away_team_player11
0,9190,9 マグノ アウベス FW,8 フェルナンジーニョ FW,10 二川 孝広 MF,7 遠藤 保仁 MF,27 橋本 英郎 MF,17 明神 智和 MF,14 家長 昭博 DF,6 山口 智 DF,2 シジクレイ DF,...,2 坪井 慶介 DF,4 田中 マルクス闘莉王 DF,20 堀之内 聖 DF,6 山田 暢久 MF,17 長谷部 誠 MF,13 鈴木 啓太 MF,8 三都主 アレサンドロ MF,18 小野 伸二 MF,10 ポンテ FW,21 ワシントン FW
1,9191,18 長谷川 太郎 FW,16 バレー FW,11 宇留野 純 FW,31 林 健太郎 MF,25 鈴木 健太 MF,8 倉貫 一毅 MF,4 山本 英臣 DF,20 ビジュ DF,15 アライール DF,...,25 市川 大祐 DF,26 青山 直晃 DF,4 高木 和道 DF,3 山西 尊裕 DF,13 兵働 昭弘 MF,7 伊東 輝悦 MF,16 枝村 匠馬 MF,10 藤本 淳吾 MF,18 チョ ジェジン FW,17 マルキーニョス FW
2,9192,9 ルーカス FW,21 ササ サルセード FW,35 リチェーリ FW,6 今野 泰幸 MF,23 梶山 陽平 MF,19 伊野波 雅彦 MF,15 鈴木 規郎 DF,5 増嶋 竜也 DF,2 茂庭 照幸 DF,...,2 三木 隆司 DF,4 深谷 友基 DF,22 上本 大海 DF,5 エジミウソン MF,11 トゥーリオ MF,6 梅田 高志 MF,17 根本 裕一 MF,26 内村 圭宏 MF,13 高松 大樹 FW,9 オズマール FW
3,9193,15 西野 泰正 FW,22 カレン ロバート FW,7 名波 浩 MF,14 村井 慎二 MF,23 福西 崇史 MF,25 ファブリシオ MF,11 西 紀寛 MF,6 服部 年宏 DF,5 田中 誠 DF,...,22 中村 北斗 DF,4 金古 聖司 DF,5 千代反田 充 DF,3 アレックス DF,7 宮崎 光平 MF,6 布部 陽功 MF,8 ホベルト MF,14 古賀 誠史 MF,27 田中 佑昌 FW,36 グラウシオ FW
4,9194,19 杉本 恵太 FW,11 玉田 圭司 FW,27 片山 奨典 MF,7 中村 直志 MF,14 吉村 圭司 MF,8 金 正友 MF,6 有村 光史 DF,29 深津 康太 DF,5 古賀 正紘 DF,...,2 ブルーノ クアドロス DF,5 前田 和哉 DF,3 柳本 啓成 DF,4 河村 崇大 MF,23 下村 東美 MF,10 ピンゴ MF,6 ゼ カルロス MF,8 森島 寛晃 FW,20 西澤 明訓 FW,9 古橋 達弥 FW
5,9195,10 マルティネス FW,8 小林 大悟 FW,11 藤本 主税 MF,16 久永 辰徳 MF,32 小林 慶行 MF,5 冨田 大介 MF,17 土屋 征夫 DF,2 奥野 誠一郎 DF,4 トニーニョ DF,...,3 斎藤 大輔 DF,5 ストヤノフ DF,2 坂本 將貴 MF,6 阿部 勇樹 MF,7 佐藤 勇人 MF,16 山岸 智 MF,8 水野 晃樹 MF,22 羽生 直剛 MF,10 ハース FW,18 巻 誠一郎 FW
6,9196,10 ジュニーニョ FW,9 我那覇 和樹 FW,11 マルクス MF,6 マルコン MF,29 谷口 博之 MF,14 中村 憲剛 MF,20 長橋 康弘 MF,2 伊藤 宏樹 DF,13 寺田 周平 DF,...,17 海本 幸治郎 DF,19 海本 慶治 DF,26 中野 洋司 DF,5 梅山 修 DF,16 寺川 能人 MF,15 本間 勲 MF,8 シルビーニョ MF,18 鈴木 慎吾 MF,10 エジミウソン FW,11 矢野 貴章 FW
7,9197,11 佐藤 寿人 FW,10 ウェズレイ FW,7 森﨑 浩司 MF,8 森﨑 和幸 MF,14 戸田 和幸 MF,6 ベット MF,17 服部 公太 DF,3 小村 徳男 DF,4 ジニーニョ DF,...,20 内田 篤人 DF,3 岩政 大樹 DF,4 大岩 剛 DF,7 新井場 徹 DF,24 青木 剛 MF,26 増田 誓志 MF,8 小笠原 満男 MF,10 本山 雅志 MF,11 深井 正樹 FW,13 柳沢 敦 FW
8,9198,19 マルケス FW,9 久保 竜彦 FW,14 奥 大介 MF,5 ドゥトラ MF,8 マグロン MF,6 上野 良治 MF,17 吉田 孝行 MF,22 中澤 佑二 DF,30 栗原 勇蔵 DF,...,4 鈴木 和裕 DF,3 リカルド DF,2 鈴木 悟 DF,7 児玉 新 DF,18 米田 兼一郎 MF,16 斉藤 大介 MF,11 星 大輔 MF,14 中払 大介 MF,10 パウリーニョ FW,31 田原 豊 FW
9,9199,21 ワシントン FW,10 ポンテ FW,18 小野 伸二 MF,8 三都主 アレサンドロ MF,13 鈴木 啓太 MF,17 長谷部 誠 MF,6 山田 暢久 MF,20 堀之内 聖 DF,4 田中 マルクス闘莉王 DF,...,2 鈴木 秀人 DF,5 田中 誠 DF,20 金 珍圭 DF,6 服部 年宏 DF,11 西 紀寛 MF,8 菊地 直哉 MF,25 ファブリシオ MF,14 村井 慎二 MF,7 名波 浩 MF,22 カレン ロバート FW


In [12]:
stadium_capacity_mapping
#使用されていないスタジアムもあるが、capacityのデータは必須だと考える

,stadium,capacity
0,日産スタジアム,72081
1,埼玉スタジアム2002,62010
2,国立競技場,54224
3,静岡スタジアムエコパ,51697
4,エコパスタジアム,51697
5,宮城スタジアム,49133
6,ひとめぼれスタジアム宮城,49133
7,味の素スタジアム,48999
8,ヤンマースタジアム長居,47853
9,大阪長居スタジアム,47853


## 特徴量作成・数値変換

In [13]:
compose_processing = compose.copy() #加工データ用にcopy

In [14]:
compose_processing.head()

,index,attendance,away_team,broadcasters,home_team,humidity,id,kick_off_time,match_date,round,section,temperature,venue,weather
0,0,20916.0,浦和,NHK総合/J SPORTS(録),G大阪,40.0,9190,16:04,2006-03-04,第1日,第1節,8.3,万博記念競技場,晴
1,1,14277.0,清水,山梨放送/テレビ静岡(録)/J SPORTS(録),甲府,28.0,9191,13:00,2006-03-05,第2日,第1節,12.9,山梨県小瀬スポーツ公園陸上競技場,晴
2,2,22531.0,大分,BS-i/MXテレビ(録)/J SPORTS(録),FC東京,35.0,9192,13:35,2006-03-05,第2日,第1節,12.1,味の素スタジアム,晴
3,3,28564.0,福岡,J SPORTS,磐田,42.0,9193,14:04,2006-03-05,第2日,第1節,11.6,静岡スタジアムエコパ,晴
4,4,17199.0,C大阪,スカイパーフェクTV!/NHK名古屋(録)/NHK大阪(録)/J SPORTS(録),名古屋,32.0,9194,14:04,2006-03-05,第2日,第1節,13.1,名古屋市瑞穂陸上競技場,晴


In [15]:
#ユニーク数カウント
at_number =compose["away_team"].drop_duplicates().nunique()
broadcasters_number = compose["broadcasters"].drop_duplicates().nunique()
ht_number = compose["home_team"].drop_duplicates().nunique()
humidity_number = compose["humidity"].drop_duplicates().nunique()


round_number = compose["round"].drop_duplicates().nunique()
section_number = compose["section"].drop_duplicates().nunique()

temp_number = compose["temperature"].drop_duplicates().nunique()
venue_number = compose["venue"].drop_duplicates().nunique()
weather_number = compose["weather"].drop_duplicates().nunique()

match_date_number = compose["match_date"].drop_duplicates().nunique()
#attendance_number = compose["attendance"].drop_duplicates().nunique()

In [16]:
#アウェイチーム名の辞書作成
away_team_dict={}
for away_team_num in range(at_number):
    away_team_dict.update({compose["away_team"].drop_duplicates().unique()[away_team_num]:away_team_num})
#置換
compose_processing["away_team"] =compose_processing["away_team"].replace(away_team_dict)

In [17]:
#放送会社の辞書の作成
broadcasters_dict={}
for broadcasters_num in range (broadcasters_number):
    broadcasters_dict.update({compose["broadcasters"].drop_duplicates().unique()[broadcasters_num]:broadcasters_num})
#置換
compose_processing["broadcasters"] =compose_processing["broadcasters"].replace(broadcasters_dict)

In [18]:
#ホームチーム名の辞書作成
home_team_dict={}
for home_team_num in range(ht_number):
    home_team_dict.update({compose["home_team"].drop_duplicates().unique()[home_team_num]:home_team_num})
#置換
compose_processing["home_team"] =compose_processing["home_team"].replace(home_team_dict)

In [19]:
#ラウンド辞書作成
round_dict={}
for round_num in range(round_number):
    round_dict.update({compose["round"].drop_duplicates().unique()[round_num]:round_num})
#置換
compose_processing["round"] =compose_processing["round"].replace(round_dict)

In [20]:
#第～節の辞書作成
section_dict ={}
for section_num in range(section_number):
    section_dict.update({compose["section"].drop_duplicates().unique()[section_num]:section_num})
#置換
compose_processing["section"] =compose_processing["section"].replace(section_dict)

In [21]:
#tcompose_processingにcapacityを追加するため
#mergeするために,列名を同じ名前にする .rename
stadium_capacity_mapping = stadium_capacity_mapping.rename(columns={"stadium":"venue"})
#mergeする
compose_processing = pd.merge(compose_processing,stadium_capacity_mapping, on ="venue")

In [22]:
#スタジアムの辞書の作成
venue_dict={}
add_venue_dict ={}
for venue_num in range(venue_number):
    venue_dict.update({compose["venue"].drop_duplicates().unique()[venue_num]:venue_num})
    add_venue_dict.update({compose["venue"].drop_duplicates().unique()[venue_num]:venue_num})

################################################################################################################
#置換
#stadium_capacity_mapping["stadium"] = stadium_capacity_mapping["stadium"].replace(add_venue_dict)
compose_processing["venue"] =compose_processing["venue"].replace(venue_dict)
########################################################################################################

In [23]:
#天気の辞書の作成
weather_dict={}
for weather_num in range (weather_number):
    weather_dict.update({compose["weather"].drop_duplicates().unique()[weather_num]:weather_num})
#置換
compose_processing["weather"] =compose_processing["weather"].replace(weather_dict)

#nanデータ　平均で置換
compose_processing["weather"]   = compose_processing["weather"].fillna(compose_processing["weather"].mean())

## 休日の追加

In [24]:
import datetime

In [25]:
compose_processing["match_date"] = pd.to_datetime(compose_processing["match_date"])
compose_processing["kick_off_time"] = pd.to_datetime(compose_processing["kick_off_time"])

In [26]:
#compose_processingにholidayを追加するため
#mergeするために,列名を同じ名前にする .rename
holidays_in_japan  = holidays_in_japan .rename(columns={"holiday_date":"match_date"})
holidays_in_japan["match_date"] = pd.to_datetime(holidays_in_japan["match_date"])

In [27]:
compose_processing = pd.merge(compose_processing,holidays_in_japan ,how = "left")

In [28]:
compose_processing.head()

,index,attendance,away_team,broadcasters,home_team,humidity,id,kick_off_time,match_date,round,section,temperature,venue,weather,capacity,description
0,0,20916.0,0,0,0,40.0,9190,2018-10-31 16:04:00,2006-03-04,0,0,8.3,0,0.0,21000,NaN
1,26,8388.0,2,7,0,79.0,9216,2018-10-31 19:04:00,2006-03-18,0,2,9.0,0,4.0,21000,NaN
2,35,9019.0,15,24,0,42.0,9225,2018-10-31 19:02:00,2006-03-29,1,3,5.9,0,1.0,21000,NaN
3,60,17967.0,7,7,0,31.0,9250,2018-10-31 19:04:00,2006-04-08,0,6,12.9,0,1.0,21000,NaN
4,73,15103.0,11,24,0,42.0,9263,2018-10-31 14:03:00,2006-04-22,0,8,15.8,0,1.0,21000,NaN


In [29]:
#ユニーク数カウント
description_number =compose_processing["description"].drop_duplicates().nunique()

In [30]:
#休日の辞書作成
holiday_dict={}
for description_num in range(description_number+1):
    holiday_dict.update({compose_processing["description"].drop_duplicates().unique()[description_num]:description_num})
#置換
compose_processing["description"] =compose_processing["description"].replace(holiday_dict)

In [32]:
holiday_dict

{nan: 0,
 'Autumnal Equinox Day': 1,
 'Showa Day': 2,
 'Childrens Day': 3,
 'Labor Thanksgiving Day': 4,
 'Culture Day': 5,
 'Greenery Day (obs)': 6,
 'Constitution Memorial Day': 7,
 'Greenery Day': 8,
 'National Holiday': 9,
 'Childrens Day (obs)': 10,
 'Marine Day': 11,
 'Vernal Equinox Day': 12}

時間　削除

In [33]:
####################################
## 列の追加
compose_processing["year"] = 0
compose_processing["month"] = 0
compose_processing["day"] = 0
################################
compose_processing["year"]  = compose_processing["match_date"].dt.year
compose_processing["month"] = compose_processing["match_date"].dt.month
compose_processing["day"]   = compose_processing["match_date"].dt.day

In [34]:
#キックオフの時間の追加
compose_processing["kick_off_hour"] =0
compose_processing["kick_off_hour"] = compose_processing["kick_off_time"].dt.hour
#######################################################################################
compose_processing["kick_off_minute"] =0
compose_processing["kick_off_minute"] =compose_processing["kick_off_time"].dt.minute

In [35]:
#曜日の追加
compose_processing["weekday"] = 0
compose_processing["weekday"] = compose_processing["match_date"].dt.dayofweek
############################################################################

In [36]:
compose_processing = compose_processing.drop("kick_off_time", axis =1)
#compose_processing = compose_processing.drop("match_date", axis =1)

# 選手の追加 ?未完成

In [37]:
match_reports

,id,home_team_player11,home_team_player10,home_team_player9,home_team_player8,home_team_player7,home_team_player6,home_team_player5,home_team_player4,home_team_player3,...,away_team_player2,away_team_player3,away_team_player4,away_team_player5,away_team_player6,away_team_player7,away_team_player8,away_team_player9,away_team_player10,away_team_player11
0,9190,9 マグノ アウベス FW,8 フェルナンジーニョ FW,10 二川 孝広 MF,7 遠藤 保仁 MF,27 橋本 英郎 MF,17 明神 智和 MF,14 家長 昭博 DF,6 山口 智 DF,2 シジクレイ DF,...,2 坪井 慶介 DF,4 田中 マルクス闘莉王 DF,20 堀之内 聖 DF,6 山田 暢久 MF,17 長谷部 誠 MF,13 鈴木 啓太 MF,8 三都主 アレサンドロ MF,18 小野 伸二 MF,10 ポンテ FW,21 ワシントン FW
1,9191,18 長谷川 太郎 FW,16 バレー FW,11 宇留野 純 FW,31 林 健太郎 MF,25 鈴木 健太 MF,8 倉貫 一毅 MF,4 山本 英臣 DF,20 ビジュ DF,15 アライール DF,...,25 市川 大祐 DF,26 青山 直晃 DF,4 高木 和道 DF,3 山西 尊裕 DF,13 兵働 昭弘 MF,7 伊東 輝悦 MF,16 枝村 匠馬 MF,10 藤本 淳吾 MF,18 チョ ジェジン FW,17 マルキーニョス FW
2,9192,9 ルーカス FW,21 ササ サルセード FW,35 リチェーリ FW,6 今野 泰幸 MF,23 梶山 陽平 MF,19 伊野波 雅彦 MF,15 鈴木 規郎 DF,5 増嶋 竜也 DF,2 茂庭 照幸 DF,...,2 三木 隆司 DF,4 深谷 友基 DF,22 上本 大海 DF,5 エジミウソン MF,11 トゥーリオ MF,6 梅田 高志 MF,17 根本 裕一 MF,26 内村 圭宏 MF,13 高松 大樹 FW,9 オズマール FW
3,9193,15 西野 泰正 FW,22 カレン ロバート FW,7 名波 浩 MF,14 村井 慎二 MF,23 福西 崇史 MF,25 ファブリシオ MF,11 西 紀寛 MF,6 服部 年宏 DF,5 田中 誠 DF,...,22 中村 北斗 DF,4 金古 聖司 DF,5 千代反田 充 DF,3 アレックス DF,7 宮崎 光平 MF,6 布部 陽功 MF,8 ホベルト MF,14 古賀 誠史 MF,27 田中 佑昌 FW,36 グラウシオ FW
4,9194,19 杉本 恵太 FW,11 玉田 圭司 FW,27 片山 奨典 MF,7 中村 直志 MF,14 吉村 圭司 MF,8 金 正友 MF,6 有村 光史 DF,29 深津 康太 DF,5 古賀 正紘 DF,...,2 ブルーノ クアドロス DF,5 前田 和哉 DF,3 柳本 啓成 DF,4 河村 崇大 MF,23 下村 東美 MF,10 ピンゴ MF,6 ゼ カルロス MF,8 森島 寛晃 FW,20 西澤 明訓 FW,9 古橋 達弥 FW
5,9195,10 マルティネス FW,8 小林 大悟 FW,11 藤本 主税 MF,16 久永 辰徳 MF,32 小林 慶行 MF,5 冨田 大介 MF,17 土屋 征夫 DF,2 奥野 誠一郎 DF,4 トニーニョ DF,...,3 斎藤 大輔 DF,5 ストヤノフ DF,2 坂本 將貴 MF,6 阿部 勇樹 MF,7 佐藤 勇人 MF,16 山岸 智 MF,8 水野 晃樹 MF,22 羽生 直剛 MF,10 ハース FW,18 巻 誠一郎 FW
6,9196,10 ジュニーニョ FW,9 我那覇 和樹 FW,11 マルクス MF,6 マルコン MF,29 谷口 博之 MF,14 中村 憲剛 MF,20 長橋 康弘 MF,2 伊藤 宏樹 DF,13 寺田 周平 DF,...,17 海本 幸治郎 DF,19 海本 慶治 DF,26 中野 洋司 DF,5 梅山 修 DF,16 寺川 能人 MF,15 本間 勲 MF,8 シルビーニョ MF,18 鈴木 慎吾 MF,10 エジミウソン FW,11 矢野 貴章 FW
7,9197,11 佐藤 寿人 FW,10 ウェズレイ FW,7 森﨑 浩司 MF,8 森﨑 和幸 MF,14 戸田 和幸 MF,6 ベット MF,17 服部 公太 DF,3 小村 徳男 DF,4 ジニーニョ DF,...,20 内田 篤人 DF,3 岩政 大樹 DF,4 大岩 剛 DF,7 新井場 徹 DF,24 青木 剛 MF,26 増田 誓志 MF,8 小笠原 満男 MF,10 本山 雅志 MF,11 深井 正樹 FW,13 柳沢 敦 FW
8,9198,19 マルケス FW,9 久保 竜彦 FW,14 奥 大介 MF,5 ドゥトラ MF,8 マグロン MF,6 上野 良治 MF,17 吉田 孝行 MF,22 中澤 佑二 DF,30 栗原 勇蔵 DF,...,4 鈴木 和裕 DF,3 リカルド DF,2 鈴木 悟 DF,7 児玉 新 DF,18 米田 兼一郎 MF,16 斉藤 大介 MF,11 星 大輔 MF,14 中払 大介 MF,10 パウリーニョ FW,31 田原 豊 FW
9,9199,21 ワシントン FW,10 ポンテ FW,18 小野 伸二 MF,8 三都主 アレサンドロ MF,13 鈴木 啓太 MF,17 長谷部 誠 MF,6 山田 暢久 MF,20 堀之内 聖 DF,4 田中 マルクス闘莉王 DF,...,2 鈴木 秀人 DF,5 田中 誠 DF,20 金 珍圭 DF,6 服部 年宏 DF,11 西 紀寛 MF,8 菊地 直哉 MF,25 ファブリシオ MF,14 村井 慎二 MF,7 名波 浩 MF,22 カレン ロバート FW


In [41]:
match_reports.home_team_player11.value_counts().head()

11 佐藤 寿人 FW      133
11 豊田 陽平 FW       90
13 大久保 嘉人 FW      88
18 前田 遼一 FW       77
18 マルキーニョス FW     73
Name: home_team_player11, dtype: int64

In [42]:
match_reports.home_team_player10.value_counts().head()

22 池田 圭 FW     53
11 玉田 圭司 FW    49
11 齋藤 学 MF     47
24 赤嶺 真吾 FW    46
13 興梠 慎三 FW    44
Name: home_team_player10, dtype: int64

同じ選手の名前がある dataframe全体で一括で名前変換するやり方不明

In [43]:
B = pd.merge(compose_processing,match_reports ,how = "left")

In [44]:
B.head()

,index,attendance,away_team,broadcasters,home_team,humidity,id,match_date,round,section,...,away_team_player2,away_team_player3,away_team_player4,away_team_player5,away_team_player6,away_team_player7,away_team_player8,away_team_player9,away_team_player10,away_team_player11
0,0,20916.0,0,0,0,40.0,9190,2006-03-04,0,0,...,2 坪井 慶介 DF,4 田中 マルクス闘莉王 DF,20 堀之内 聖 DF,6 山田 暢久 MF,17 長谷部 誠 MF,13 鈴木 啓太 MF,8 三都主 アレサンドロ MF,18 小野 伸二 MF,10 ポンテ FW,21 ワシントン FW
1,26,8388.0,2,7,0,79.0,9216,2006-03-18,0,2,...,2 三木 隆司 DF,22 上本 大海 DF,31 福元 洋平 DF,6 梅田 高志 MF,11 トゥーリオ MF,5 エジミウソン MF,7 西山 哲平 MF,17 根本 裕一 MF,13 高松 大樹 FW,9 オズマール FW
2,35,9019.0,15,24,0,42.0,9225,2006-03-29,1,3,...,22 波戸 康広 DF,4 トニーニョ DF,2 奥野 誠一郎 DF,17 土屋 征夫 DF,6 ディビッドソン 純マーカス MF,19 片岡 洋介 MF,8 小林 大悟 MF,11 藤本 主税 MF,14 森田 浩史 FW,30 桜井 直人 FW
3,60,17967.0,7,7,0,31.0,9250,2006-04-08,0,6,...,20 内田 篤人 DF,3 岩政 大樹 DF,4 大岩 剛 DF,7 新井場 徹 DF,26 増田 誓志 MF,24 青木 剛 MF,16 フェルナンド MF,8 小笠原 満男 MF,10 本山 雅志 FW,9 アレックス ミネイロ FW
4,73,15103.0,11,24,0,42.0,9263,2006-04-22,0,8,...,25 徳永 悠平 DF,3 ジャーン DF,2 茂庭 照幸 DF,15 鈴木 規郎 DF,6 今野 泰幸 MF,23 梶山 陽平 MF,16 宮沢 正史 MF,27 栗澤 僚一 MF,9 ルーカス FW,20 川口 信男 FW


In [45]:
compose_processing.shape

(3843, 21)

In [46]:
compose_processing.head()

,index,attendance,away_team,broadcasters,home_team,humidity,id,match_date,round,section,...,venue,weather,capacity,description,year,month,day,kick_off_hour,kick_off_minute,weekday
0,0,20916.0,0,0,0,40.0,9190,2006-03-04,0,0,...,0,0.0,21000,0,2006,3,4,16,4,5
1,26,8388.0,2,7,0,79.0,9216,2006-03-18,0,2,...,0,4.0,21000,0,2006,3,18,19,4,5
2,35,9019.0,15,24,0,42.0,9225,2006-03-29,1,3,...,0,1.0,21000,0,2006,3,29,19,2,2
3,60,17967.0,7,7,0,31.0,9250,2006-04-08,0,6,...,0,1.0,21000,0,2006,4,8,19,4,5
4,73,15103.0,11,24,0,42.0,9263,2006-04-22,0,8,...,0,1.0,21000,0,2006,4,22,14,3,5


In [47]:
compose_processing.to_csv("compose_processing.csv",header=True,index=None)